In [1]:
import ROOT as rt
# import root_numpy as rtnp
import csv
import re
import sys
import collections

from collections import OrderedDict
import uproot
import pandas as pd

import scipy
import awkward
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
sys.path.append('/storage/user/christiw/gpu/christiw/llp/delayed_jet_analyzer/lib/')
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot

import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 0

wH = 1
Z_MASS = 91.2


# donotdelete = []
print(sys.version)

Welcome to JupyROOT 6.18/02
3.6.8 (default, Aug  7 2019, 17:28:10) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-39)]


# Load ntuples

In [2]:
fpath_bkg =OrderedDict()
tree_bkg = OrderedDict()
tree_sig = OrderedDict()
fpath_sig =OrderedDict()


start_t = time.time()
data_year = 'full'
pdgId = 13
category = 0
OLD_CTAU = 100 #cm
ntupler_version = 'V1p14/'
analyzer_version = 'v1/v1/'

# ntupler_version = 'V1p12/'
# analyzer_version = 'v4/v4/'
if data_year == 2016:
    data_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p12/Data'+str(data_year)+'/v4/v4/normalized/'
elif data_year == 2017:
    data_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p12/Data'+str(data_year)+'/v4/v4/normalized/'
elif data_year == 2018:
    data_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p12/Data'+str(data_year)+'/v4/v4/normalized/'
elif data_year == 'full':
    data_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p12/Data2018/v4/v4/normalized/'

else:
    print("DATA YEAR IS WRONG")
    raise NameError('DATA YEAR IS WRONG')
mc_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_RunIIFall18/'+analyzer_version+'/normalized/'



if category == 0:
    if data_year == 2016:
        fpath_bkg['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p12_Data2016_Run2016-HighMET-07Aug17_goodLumi.root'
        fpath_bkg['mc'] = mc_path + 'ggH_HToSSTobbbb_ms55_pl1000_RunIIFall18_1pb_weighted.root'
        lumi = (5.632467289+2.572903489+1.685313225+3.988453305+3.068943620+3.527270568+8.609721915)*1000 #29.085073411
    elif data_year == 2017:
        fpath_bkg['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p12_Data2017_Run2017-HighMET-17Nov2017_goodLumi.root'
        fpath_bkg['mc'] = mc_path + 'ggH_HToSSTobbbb_ms55_pl1000_RunIIFall18_1pb_weighted.root'
        lumi = (4.738190514+4.145487893+ 9.295575564+2.603562425) * 1000 #20.782816396
 
    elif data_year == 2018:
        fpath_bkg['mc'] = mc_path + 'ggH_HToSSTobbbb_ms55_pl1000_RunIIFall18_1pb_weighted.root'

        fpath_bkg['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p12_Data2018_17Sept2018_Run2018-HighMET-17Sep2018_goodLumi.root'
        lumi = (13.954129666 + 6.941561868 + 3.227904890 + 31.312400879) * 1000 #55.435997303
#         fpath_bkg['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p12_Data2018_17Sept2018_Run2018AB-HighMET-17Sep2018_goodLumi.root'
#         lumi = (13.954129666 + 6.941561868 ) * 1000 #AB
#         fpath_bkg['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p12_Data2018_17Sept2018_Run2018CD-HighMET-17Sep2018_goodLumi.root'
#         lumi = (3.227904890 + 31.312400879) * 1000 #CD
    else: #2016, 2017, 2018AB
        if OLD_CTAU == 100:
            fpath_bkg['mc'] = mc_path + 'ggH_HToSSTobbbb_ms55_pl1000_1pb_weighted.root'
        else:
            fpath_bkg['mc'] = mc_path + 'ggH_HToSSTobbbb_ms55_pl10000_1pb_weighted.root'
            #         fpath_bkg['mc'] = mc_path + 'ggH_HToSSTobbbb_ms55_pl1000_RunIIFall18_1pb_weighted.root'
        fpath_bkg['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p12_Data2016_Data2017_Data2018-HighMET_goodLumi.root'

        lumi = (29.085073411 + 20.782816396+55.435997303)* 1000


elif category == 1:
    fpath_bkg['mc'] = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p12/MC_RunIIFall18/v3/v5/normalized/WH_HToSSTobbbb_ms55_pl10000_1pb_weighted.root'

#     if pdgId == 13:
#         fpath_bkg['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p12_Data2018_SingleMuon_17Sept2018_Run2018D-ZMu-PromptReco-v2.root'
#     else:
#         fpath_bkg['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p12_Data2018_EGamma_17Sept2018_Run2018D-ZElectron-PromptReco-v2.root'
    lumi = 31.34*1000
else:
    data_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p12/Data'+str(data_year)+'/v2/v3/normalized/'
    mc_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p12/MC_RunIISummer16/v1/v2/signals/normalized/'
    fpath_bkg['mc'] = mc_path + 'ZH_HToSSTobbbb_ms55_pl1000_1pb_weighted.root'
    fpath_bkg['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p12_Data2018_SingleMuon_17Sept2018_Run2018D-ZMu-PromptReco-v2_goodLumi.root'
    lumi = 29.516263349*1000

NEvents = {}
print(data_year, lumi/1000)
NEvents_genweight = {}
for k,v in fpath_bkg.items():
    print (k, v)
#     tree[k] = rtnp.root2array(v)
    root_dir = uproot.open(v) 
#     if k[:7] == 'ntuples':
#         tree_bkg[k] = root_dir['ntuples']['llp']
#         NEvents[k] = root_dir['ntuples']['NEvents'][1]
#     else:
    tree_bkg[k] = root_dir['MuonSystem']
    NEvents[k] = root_dir['NEvents'][1]
    NEvents_genweight[k] = root_dir['NEvents_genweight'][1]
#     tree[k] = root_dir['ntuples']
    a = tree_bkg[k]["weight"].array()
    print("NEvents",NEvents[k])
    print("NEvents_genweight",NEvents_genweight[k])


    print ('weights',np.unique(a)*lumi)
    print(np.sum(a))
    print(len(a))
#     print(len(a)/ NEvents[k])
#     print(a.shape)
print(time.time()-start_t)

here
full 105.30388711
mc /mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p14//MC_RunIIFall18/v1/v1//normalized/ggH_HToSSTobbbb_ms55_pl1000_1pb_weighted.root
NEvents 1285000.0
NEvents_genweight 27594916.0
weights [-4.0220227  3.9419844  3.9818022  4.0220227]
0.39818925
10443
data /mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p12/Data2018/v4/v4/normalized/Run2_displacedJetMuonNtupler_V1p12_Data2016_Data2017_Data2018-HighMET_goodLumi.root
NEvents 16130933.0
NEvents_genweight 0.0
weights [105303.89]
14992230.0
14992230
361.3488755226135


# Load trigger

In [3]:
trigger_names_file = '/storage/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/data/trigger_names_llp_v1.dat'
trigger_names = []
with open(trigger_names_file) as f:
    reader = csv.reader(f, delimiter=" ")
    for line in reader:
        trigger_names.append(line[2])
# if wH:
#     trigger_paths = [87,135,136] #PFMET120
# #     elif data_year == 2017: trigger_paths = [87,136] #PFMET120
# else:
trigger_paths = [177,362,87,135,136] #PFMET120
trigger_paths += [84,91]
if category == 0:
    trigger_paths = [310]
xsec = {}
xsec['bbbb10m'] = (5.328E-01 + 8.4E-01) *0.324
xsec['bbbb1m'] = (5.328E-01 + 8.4E-01) *0.324
xsec['bbbb0.1m'] = (5.328E-01 + 8.4E-01) *0.324


xsec['WJetsToLNu'] = 61526.7

legend = {}
legend['bbbb10m'] = 'signal c#tau = 10 m'
legend['bbbb1m'] = 'signal c#tau = 1 m'
legend['bbbb0.1m'] = 'signal c#tau = 0.1 m'

legend['SingleElectron'] = 'SingleElectron'
legend['SingleMuon'] = 'SingleMuon'
legend['WJetsToLNu'] = 'WJetsToLNu'

# load bdt

In [4]:
# make predictions for test data

import pickle
# model = pickle.load(open( 'bdt_flatten_noEvtSelection.pickle.dat', "rb" ))
bdt_name = 'bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs_EtaPhi'
model = pickle.load(open( bdt_name+'.pickle', "rb" ))
y_pred = model.predict_proba([1,2,3,4,4,5,6])[:, 1]
print (y_pred)


[0.283564]


In [5]:
import math
def deltaPhi( phi1,  phi2):
    dphi = phi1-phi2
    while (dphi > math.pi):
        dphi -= 2*math.pi
    while (dphi <= -math.pi):
        dphi += 2*math.pi
    return dphi
def deltaR(eta1, phi1, eta2, phi2):
    dphi = deltaPhi(phi1,phi2)
    deta = eta1 - eta2
    return (dphi*dphi + deta*deta)**0.5

# nCsc with different hit vetoing

In [6]:
%%time

start_t = time.time()
JET_PT_CUT = 10.0
MUON_PT_CUT = 20.0
N_RECHIT_CUT = 90
jetPt_cut = 50
tightid = False


# BDT_CUT = 0.467 #90% wp vBDT/v1, bdt_flatten_noEvtSelection
# bdtBkgEff =  0.0661076084960519

# BDT_CUT = 0.9227871 #90% wp vBDT/v2, bdt_flatten_metfilter_me1112nohits_eta2p1
# bdtBkgEff =  0.10278372591006424
if bdt_name == 'bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs':
    BDT_CUT = 0.92245656 #bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs
    bdtBkgEff = 0.09957173447537473
elif bdt_name == 'bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs_EtaPhi':
    BDT_CUT = 0.9608465 #bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs_EtaPhi
    bdtBkgEff = 0.06745182012847965
else:
    print('BDT NAME NOT FOUND')

intime = True
DPHI_CUT = 1
weight = {}
weight_event = {}
weight_ctau = {}
lumiSec = {}
evtNum = {}
nCsc = {}
npv = {}
npu = {}
runNum = {}
nCsc_JetMuonVetoCluster0p4_Me1112Veto = {}
gLLP_csc = {}
cscClusterSize = {}
cscClusterTime = {}
nCscClusters = {}
selections_cluster = {}
sel_cluster = {}
sel_jetveto = {}
met_trigger = {}
met = {}
gLLP_beta = {}
jetPt = {}
jetPhi = {}
metPhi = {}
angle ={}
nLeptons = {}
genJetPt = {}
genJetPhi = {}
genMet = {}
genMetPhi = {}
pileupWeight = {}
gLLP_ctau = {}
npv = {}
nRechitClusters = {}
nJets = {}
nJets_50gev = {}
cscRechitClusterTimeDiff = {}
cscRechitCluster_match_gLLP = {}

cscRechitClusterXSpread = {}
cscRechitClusterYSpread = {}
cscRechitClusterNStation = {}
cscRechitClusterEtaPhiSpread = {}
cscRechitClusterPhiSpread = {}
cscRechitClusterEtaSpread = {}
cscRechitClusterX = {}
cscRechitClusterY = {}
cscRechitClusterZ = {}
cscRechitClusterPhi = {}
cscClusterJetVetoPt = {}
cscRechitClusterEta = {}

cscRechitClusterMaxStationRatio = {}
cscRechitClusterNStation = {}
cscRechitClusterNChamber = {}
cscRechitClusterMet_dPhi = {}
jetMet_dPhiMin30 = {}
jetMet_dPhiMin = {}
dphiMet_cluster = {}
nRechits_sr = {}
jetMet_dPhiMin30_sr = {}
bdt_score = {}
a = {}
b = {}
c = {}
d = {}
sel_ev = {}
bdt_sel = {}

legend = {}

legend['data_oot'] = 'Data OOT region'
legend['data_intime'] = 'Data in-time region'
legend['mc_signal'] = 'signal MC in time'
legend['mc_intime'] = 'MC in-time background'
legend['mc_oot'] = 'MC OOT background'
legend['mc_bkg'] = 'QCD 50toInf background'

# keys = ['data','mc_intime_bkg','mc_oot_bkg','mc_signal']
keys = ['data_intime','data_oot','mc_signal','mc_bkg']
keys = ['data_intime','mc_signal',]
keys = ['data_intime','mc_signal']
keys = ['data_vr','mc_vr','mc_sr']

for k in keys:
    print(k)
#     if k == 'data_intime': continue
    if k == 'mc_bkg':
        T = tree_bkg['QCDHT50toInf']
    elif k[:2] == 'mc':
        T = tree_bkg['mc']
    else:
        T = tree_bkg['data']

        
########### SELECTION: CLUSTERS ############

    sel_rechitcluster =  np.abs(T.array('cscRechitClusterMaxChamber')) > 12
    sel_rechitcluster = np.logical_and(sel_rechitcluster, np.abs(T.array('cscRechitClusterEta')) < 2.1)

    me1112_veto = 0
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitClusterNRechitChamberPlus11') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitClusterNRechitChamberPlus12') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitClusterNRechitChamberMinus11') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitClusterNRechitChamberMinus12') <= me1112_veto)
    np.set_printoptions(threshold=10000)
    print("rechit cluster", np.count_nonzero(sel_rechitcluster.flatten()))
    if intime:
        sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitClusterJetVetoPt') < JET_PT_CUT)
        sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitClusterMuonVetoPt') < MUON_PT_CUT)
        sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_and(T.array('cscRechitClusterTime') < 12.5, T.array('cscRechitClusterTime') > -5.0))
    else:
        sel_rechitcluster = np.logical_and(sel_rechitcluster,  T.array('cscRechitClusterTime') < -12.5)

########### SELECTION: JETS ############
    
    sel_jet = np.logical_and(T.array('jetPt') > jetPt_cut, np.abs(T.array('jetEta')) < 2.4 )

########### SELECTION: EVENTS ############
    hlt = T['HLTDecision'].array()
    # select only triggered events
    sel_ev[k] = np.ones(hlt[:,0].shape, dtype=bool)
    sel_ev[k] = np.zeros(hlt[:,0].shape)
    for tr in trigger_paths:            
        sel_ev[k]   = np.logical_or(sel_ev[k] ,hlt[:,tr])
        print("trigger",tr,1.0*np.count_nonzero(hlt[:,tr])/len(hlt[:,tr]))
    sel_ev[k]  = np.logical_and(sel_ev[k], T.array('category') == category)
    sel_ev[k]  = np.logical_and(sel_ev[k],sel_rechitcluster.sum() == 1)

    if category == 0:
        sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('met') > 200)
        sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('nLeptons') == 0)
        sel_ev[k] = np.logical_and(sel_ev[k] , sel_jet.sum()>=1)

        if k[:4] == 'data':
            sel_ev[k] = np.logical_and(sel_ev[k], T.array('Flag_HBHENoiseFilter'))
            sel_ev[k] = np.logical_and(sel_ev[k], T.array('Flag_HBHEIsoNoiseFilter'))
            sel_ev[k] = np.logical_and(sel_ev[k], T.array('Flag_BadPFMuonFilter'))
            sel_ev[k] = np.logical_and(sel_ev[k], T.array('Flag_CSCTightHaloFilter'))
            sel_ev[k] = np.logical_and(sel_ev[k], T.array('Flag_goodVertices'))
#             sel_ev = np.logical_and(sel_ev, T.array('runNum')<319077)
#             print('selev',np.count_nonzero(sel_ev))
#             sel_ev = np.logical_and(sel_ev, T.array('Flag_ecalBadCalibFilter'))
#             print('selev',np.count_nonzero(T.array('Flag_ecalBadCalibFilter')))
    gLLP_csc[k] = T.array('gLLP_csc')
    if k[:2] == 'mc':
        sel_ev[k] = np.logical_and(sel_ev[k], np.sum(T.array('gLLP_csc'),axis = 1) > 0)
########### BRANCHES ############

   ##### event variables ##### 
    
    gLLP_beta[k] = T.array('gLLP_beta')[sel_ev[k]]


    ##### bdt variables ####
    
    cscRechitClusterPhi[k] = T.array('cscRechitClusterPhi')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterXSpread[k] = T.array('cscRechitClusterXSpread')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterYSpread[k] = T.array('cscRechitClusterYSpread')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterEtaSpread[k] = T.array('cscRechitClusterEtaSpread')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterPhiSpread[k] = T.array('cscRechitClusterPhiSpread')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterNStation[k] = T.array('cscRechitClusterNStation')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterEtaPhiSpread[k] = T.array('cscRechitClusterEtaPhiSpread')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterX[k] = T.array('cscRechitClusterX')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterY[k] = T.array('cscRechitClusterY')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterZ[k] = T.array('cscRechitClusterZ')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterEta[k] = T.array('cscRechitClusterEta')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterPhi[k] = T.array('cscRechitClusterPhi')[sel_rechitcluster][sel_ev[k]][:,0].flatten()

    if bdt_name == 'bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs':
        
        bdt_var = {'cscRechitClusterXSpread': cscRechitClusterXSpread[k],
         'cscRechitClusterYSpread': cscRechitClusterYSpread[k],
         'cscRechitClusterNStation':cscRechitClusterNStation[k],
         'cscRechitClusterEtaPhiSpread':cscRechitClusterEtaPhiSpread[k],
         'cscRechitClusterX':cscRechitClusterX[k],
         'cscRechitClusterY':cscRechitClusterY[k],
         'cscRechitClusterZ':cscRechitClusterZ[k],
        }
    elif bdt_name == 'bdt_flatten_metfilter_me1112nohits_eta2p1_testsize0p2_abs_EtaPhi':
        bdt_var = {'cscRechitClusterNStation': cscRechitClusterNStation[k],
         'cscRechitClusterEtaSpread': cscRechitClusterEtaSpread[k],
         'cscRechitClusterPhiSpread':cscRechitClusterPhiSpread[k],
         'cscRechitClusterEtaPhiSpread':cscRechitClusterEtaPhiSpread[k],
         'cscRechitClusterPhi':cscRechitClusterPhi[k],
         'cscRechitClusterEta':np.abs(cscRechitClusterEta[k]),
         'cscRechitClusterZ':np.abs(cscRechitClusterZ[k]),
        }
    else:
        print('BDT NAME NOT FOUND')
    
    
    dataset = pd.DataFrame(bdt_var)
    bdt_score[k] = model.predict_proba(dataset.values)[:, 1]
    print(len(bdt_score[k]) == np.count_nonzero(sel_ev[k]))
    bdt_sel[k] = bdt_score[k] < BDT_CUT 
    if 'vr' in k:
        bdt_sel[k] = bdt_score[k] < BDT_CUT 
    else:
        bdt_sel[k] = bdt_score[k] >= BDT_CUT 
    print("effiency",np.count_nonzero(bdt_sel[k])/len(bdt_sel[k]))



    dphiMet_cluster[k] = np.abs(T.array('cscRechitClusterMet_dPhi'))[sel_rechitcluster][sel_ev[k]][:,0].flatten()[bdt_sel[k]]

    jetMet_dPhiMin[k] = T.array('jetMet_dPhiMin')[sel_ev[k]][bdt_sel[k]]
    metPhi[k] = T.array('metPhi')[sel_ev[k]]
    jetPhi[k] = T.array('jetPhi')[sel_ev[k]]
    jetPt[k] = T.array('jetPt')[sel_ev[k]]
    jetMet_dPhiMin30[k] = []
    for ev in range(len(jetPhi[k])):
        dphi_min_temp = 999
        for i in range(len(jetPhi[k][ev])):
            if jetPt[k][ev][i] < 30: continue
            dphi_temp = abs(deltaPhi(metPhi[k][ev],jetPhi[k][ev][i]))
            if dphi_min_temp > dphi_temp: dphi_min_temp = dphi_temp
        jetMet_dPhiMin30[k].append(dphi_min_temp)
    jetMet_dPhiMin30[k] = np.array(jetMet_dPhiMin30[k])
    jetMet_dPhiMin30_sr[k] = jetMet_dPhiMin30[k][np.logical_not(bdt_sel[k])]
    jetMet_dPhiMin30[k] = jetMet_dPhiMin30[k][bdt_sel[k]]
    weight[k] = T.array('weight')[sel_ev[k]][bdt_sel[k]]
    npv[k] = T.array('npv')[sel_ev[k]][bdt_sel[k]]

    
    if k[:2] == 'mc':
        pileupWeight[k] = T.array('pileupWeight')[sel_ev[k]][bdt_sel[k]]
    else:
        pileupWeight[k] = np.ones(weight[k].shape, dtype=bool)
    if k[:2] == 'mc':
        weight[k] = weight[k]*lumi
    ##### clusters #####
    cscClusterTime[k] = T.array('cscRechitClusterTime')[sel_rechitcluster][sel_ev[k]][bdt_sel[k]][:,0]
    cscClusterSize[k] =  T.array('cscRechitClusterSize')[sel_rechitcluster][sel_ev[k]][bdt_sel[k]]
    nRechits_sr[k] =  T.array('cscRechitClusterSize')[sel_rechitcluster][sel_ev[k]][np.logical_not(bdt_sel[k])]

    nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] = cscClusterSize[k][:,0]
   
    br = 1
    var = jetMet_dPhiMin30[k]
    print(br)
    print("using dPhi(jet,met)")
    DPHI_CUT = 0.6
    a[k] = br * np.sum(weight[k][np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var>=DPHI_CUT)])
    b[k] = br * np.sum(weight[k][np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var>=DPHI_CUT)])
    c[k] = br * np.sum(weight[k][np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var<DPHI_CUT)])
    d[k] = br * np.sum(weight[k][np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var<DPHI_CUT)])
    print(a[k],b[k],c[k],d[k])
    print(b[k]/c[k]*d[k],a[k])
    print("uncertainty on a: ", (1./b[k] + 1./c[k] + 1./d[k])**0.5*(b[k]/c[k]*d[k]))
    if k == 'mc_sr':
        sig_dphiJet = a[k]
    elif k == 'data_vr':
        bkg_dphiJet = a[k]/(1-bdtBkgEff)*bdtBkgEff
    var = dphiMet_cluster[k]
    DPHI_CUT = 0.75
    a[k] = br * np.sum(weight[k][np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var>=DPHI_CUT)])
    b[k] = br * np.sum(weight[k][np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var>=DPHI_CUT)])
    c[k] = br * np.sum(weight[k][np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var<DPHI_CUT)])
    d[k] = br * np.sum(weight[k][np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var<DPHI_CUT)])
    if k == 'mc_sr':
        sig_dphiCluster = d[k]
    elif k == 'data_vr':
        bkg_dphiCluster = a[k]/(1-bdtBkgEff)*bdtBkgEff
    
#     if k == 'data_intime':
#         print(np.sum(weight[k][nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<70]/(1-bdtBkgEff)*bdtBkgEff))
#         print(np.count_nonzero(nRechits_SR[k]<70))
    

# print("branching ratio: "+str(br))
# print(sig_dphiJet*137000/lumi, bkg_dphiJet*137000/lumi)
# print(sig_dphiCluster*137000/lumi, bkg_dphiCluster*137000/lumi)

# br = 0.005
# print("branching ratio: "+str(br))
# print("asimov dataset")
# s = br*sig_dphiJet*137000/lumi
# b = bkg_dphiJet* 137000/lumi
# sig = (2*(s+b)*math.log(1+s/b)-s)**0.5
# print("significance for dPhi(jet, MET): ", sig)
# s = br*sig_dphiCluster*137000/lumi
# b = bkg_dphiCluster* 137000/lumi
# sig = (2*(s+b)*math.log(1+s/b)-s)**0.5
# print("significance for dPhi(cluster,MET): ", sig)


data_vr
rechit cluster 83130


/usr/local/lib/python3.6/site-packages/awkward/array/jagged.py:1031: RuntimeWarning: invalid value encountered in less
  result = getattr(ufunc, method)(*inputs, **kwargs)


trigger 310 0.744591631798605
True
effiency 0.9020452099031216
1
using dPhi(jet,met)
32.0 256.0 485.0 65.0
34.30927835051546 32.0
uncertainty on a:  5.013467415747312
mc_vr
rechit cluster 767
trigger 310 0.9704108014938236
True
effiency 0.06262626262626263
1
using dPhi(jet,met)
88.4845199584961 36.198204040527344 0.0 0.0
nan 88.4845199584961
uncertainty on a:  nan
mc_sr


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:272: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:272: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:273: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:273: RuntimeWarning: invalid value encountered in double_scalars


rechit cluster 767
trigger 310 0.9704108014938236
True
effiency 0.9373737373737374
1
using dPhi(jet,met)
1475.841064453125 172.94696044921875 8.044045448303223 201.1011199951172
4323.67366266254 1475.841064453125
uncertainty on a:  1589.0327214039878
CPU times: user 1min 45s, sys: 11.4 s, total: 1min 56s
Wall time: 4min 36s


# life time reweight

In [12]:
 #in cm
ctaus = ['5','10','30','40', '50','100','200','300','500','1000', '5000', '10000', '100000']
k = 'mc_vr'
T = tree_bkg['mc']
for ctau in ctaus:
    new_ctau = float(ctau)
    gLLP_ctau[ctau] = np.sum(T.array('gLLP_ctau'), axis = 1)[sel_ev[k]][bdt_sel[k]]
    weight_ctau[ctau] = np.exp(gLLP_ctau[ctau]*(1.0/OLD_CTAU - 1.0/new_ctau))*(OLD_CTAU*1.0/new_ctau)**2

# Signal Contamination

In [15]:
 #in cm
ctaus = ['5','10','30','40', '50','100','200','300','500','1000', '5000', '10000', '100000']
ctaus = ['100']
k = 'mc_signal'
T = tree_bkg['mc']
br = 0.01 # use 1%
DPHI_CUT = 0.6
N_RECHIT_CUTS = np.arange(120,220, 10)

for N_RECHIT_CUT in N_RECHIT_CUTS:
    a_data = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto['data_vr'] >= N_RECHIT_CUT, jetMet_dPhiMin30['data_vr'] >= DPHI_CUT))
    b_data = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto['data_vr'] < N_RECHIT_CUT, jetMet_dPhiMin30['data_vr'] >= DPHI_CUT))
    c_data = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto['data_vr'] < N_RECHIT_CUT, jetMet_dPhiMin30['data_vr'] < DPHI_CUT))
    d_data = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto['data_vr'] >= N_RECHIT_CUT, jetMet_dPhiMin30['data_vr'] < DPHI_CUT))
    for ctau in ctaus:
        new_ctau = float(ctau)
        w = br * pileupWeight['mc_vr'] * weight['mc_vr'] * weight_ctau[ctau]
        a = np.sum(w[np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto['mc_vr']>=N_RECHIT_CUT, jetMet_dPhiMin30['mc_vr']>=DPHI_CUT)])
        b = np.sum(w[np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto['mc_vr']<N_RECHIT_CUT, jetMet_dPhiMin30['mc_vr']>=DPHI_CUT)])
        c = np.sum(w[np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto['mc_vr']<N_RECHIT_CUT, jetMet_dPhiMin30['mc_vr']<DPHI_CUT)])
        d = np.sum(w[np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto['mc_vr']>=N_RECHIT_CUT, jetMet_dPhiMin30['mc_vr']<DPHI_CUT)])

        print(N_RECHIT_CUT, a/a_data, a, a_data)
        w = br * pileupWeight['mc_sr'] * weight['mc_sr'] 
        a = np.sum(w[np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto['mc_sr']>=N_RECHIT_CUT, jetMet_dPhiMin30['mc_sr']>=DPHI_CUT)])
#         print(a)
        
        
#         print(N_RECHIT_CUT, a/a_data,b/b_data,c/c_data,d/d_data)

120 0.07240222891171773 0.86882675 12
130 0.12411810670580183 0.86882675 7
140 0.1184875454221453 0.8294128 7
150 0.13799285888671875 0.6899643 5
160 0.17249107360839844 0.6899643 4
170 0.1632307469844818 0.652923 4
180 0.1632307469844818 0.652923 4
190 0.21764099597930908 0.652923 3
200 0.5789625644683838 0.57896256 1
210 0.5789625644683838 0.57896256 1


# check bdt eff in low N_rechits bins

In [270]:
# k = 'data_intime'
# print("n_rechit, VR, SR, unc")
# for n_rechits in [60,70,80,90]:
#     for dphi in [ 0.4]:
# #         print("dphi, n_rechits", dphi, n_rechits)
#         inverted = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<n_rechits, jetMet_dPhiMin30[k]<dphi))
#         inverted_unc = inverted**0.5/(1-bdtBkgEff)*bdtBkgEff
#         inverted = inverted/(1-bdtBkgEff)*bdtBkgEff
#         print(n_rechits, np.count_nonzero(np.logical_and(nRechits_SR[k]<n_rechits, jetMet_dPhiMin30_sr[k]<dphi)), round(inverted,2), round(inverted_unc,2))

# make datacard

In [284]:
np.arange(120,220, 10)

array([120, 130, 140, 150, 160, 170, 180, 190, 200, 210])

In [11]:
import os

N_RECHIT_CUTS = np.arange(120,220, 10)
# N_RECHIT_CUT = 120
DPHI_CUT = 0.6
UNBLIND = False

for N_RECHIT_CUT in N_RECHIT_CUTS:


    k = 'data_vr'

    a = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, jetMet_dPhiMin30[k] >= DPHI_CUT))/(1-bdtBkgEff)*bdtBkgEff
    b = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < N_RECHIT_CUT, jetMet_dPhiMin30[k] >= DPHI_CUT))/(1-bdtBkgEff)*bdtBkgEff
    c = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < N_RECHIT_CUT, jetMet_dPhiMin30[k] < DPHI_CUT))/(1-bdtBkgEff)*bdtBkgEff
    d = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, jetMet_dPhiMin30[k] < DPHI_CUT))/(1-bdtBkgEff)*bdtBkgEff
    c1 = b/c
    c2 = d/c
    a_pred = b*d/c
    stat_unc = (1./b + 1./c + 1./d)**0.5*(b/c*d)
    sys_unc = abs(a-a_pred)/a_pred
    unc = [(stat_unc**2 + sys_unc**2)**0.5, b**0.5, c**0.5, d**0.5]
    print(a, b, c, d, a_pred)
    
#     print(a*(1-bdtBkgEff)/bdtBkgEff, b*(1-bdtBkgEff)/bdtBkgEff, c*(1-bdtBkgEff)/bdtBkgEff, d*(1-bdtBkgEff)/bdtBkgEff, a_pred*(1-bdtBkgEff)/bdtBkgEff)

    #####
    # a = c*c1*c2
    # b = c1* c
    # c = c
    # d = c2*c
    #####
    k = 'mc_sr'
#     br = 0.001 # 0.01 percent br
    # ntupler_version = 'V1p12'
    # analyzer_version = '/v4/v4/'
    outDataCardsDir = '/storage/user/christiw/login-1/christiw/LLP/CMSSW_10_2_13/src/HiggsAnalysis/MuonSystemLimit/combine/datacards/'+ntupler_version+analyzer_version
    if not os.path.isdir(outDataCardsDir):os.mkdir(outDataCardsDir)
    sig_norm = []

    for ctau in ctaus:
        new_ctau = int(ctau)
        modelName = 'ggh_mh125_mx55_ctau'+str(int(OLD_CTAU/100))+'m_reweight_'+str(new_ctau)+'cm_nRechit'+str(N_RECHIT_CUT)+'dPhi0p6'
        signal_rate = []
        w = pileupWeight[k] * weight[k] * weight_ctau[ctau]
        cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, jetMet_dPhiMin30[k]>=DPHI_CUT)
        signal_rate.append(np.sum(w[cond]))
        
        cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, jetMet_dPhiMin30[k]>=DPHI_CUT)
        signal_rate.append(np.sum(w[cond]))
        cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, jetMet_dPhiMin30[k]<DPHI_CUT)
        signal_rate.append(np.sum(w[cond]))
        cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, jetMet_dPhiMin30[k]<DPHI_CUT)
        signal_rate.append(np.sum(w[cond]))
#         print(np.sum(signal_rate), np.sum(w))
#         print(ctau,N_RECHIT_CUT, signal_rate)
#         print(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k][nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=120])
        norm = np.sum(signal_rate)/2
    #     norm= signal_rate[0]/1.57
    #     norm = np.min(signal_rate)/0.1
        sig_norm.append(norm)
        if new_ctau == 100: print("signal", signal_rate)
        signal_rate = signal_rate/norm
        
    #     signal_rate = signal_rate/signal_rate[0]
        make_datacard(outDataCardsDir, modelName, signal_rate, norm, a_pred,b,c,d, unc)
#         if new_ctau == 100: print("signal", signal_rate)
#     print([int(a) for a in ctaus])
# print(sig_norm)


0.8679678530424798 19.963260619977035 38.55223880597014 1.2296211251435132 0.6367268865658717
signal [1473.9126, 279.7896, 29.397078, 171.97713]
0.5063145809414465 20.32491389207807 38.69690011481056 1.0849598163030996 0.5698573988433102
signal [1440.279, 313.42297, 33.644295, 167.7299]
0.5063145809414465 20.32491389207807 39.203214695752 0.5786452353616532 0.29999872903436264
signal [1403.3794, 350.3227, 42.50543, 158.86877]
0.36165327210103326 20.469575200918484 39.27554535017221 0.5063145809414465 0.2638803432899252
signal [1373.2905, 380.4115, 46.78697, 154.58723]
0.2893226176808266 20.541905855338687 39.42020665901262 0.36165327210103326 0.1884578518838412
signal [1344.4524, 409.2498, 46.78697, 154.58723]
0.2893226176808266 20.541905855338687 39.56486796785304 0.21699196326061995 0.112661275257799
signal [1304.9998, 448.70242, 50.4911, 150.88312]
0.2893226176808266 20.541905855338687 39.637198622273246 0.1446613088404133 0.0749704593260536
signal [1273.4146, 480.2876, 57.195545, 1

In [10]:
def make_datacard(outDataCardsDir,modelName, signal_rate, norm, a,b,c,d, unc):
    c1 = b/c
    c2 = d/c
    
    text_file = open(outDataCardsDir+modelName+".txt", "w")
    text_file.write('# signal norm {0} \n'.format(norm))

    text_file.write('imax {0} \n'.format(4))
    text_file.write('jmax {0} \n'.format(1))
    text_file.write('kmax * \n')
    text_file.write('--------------- \n')
    text_file.write('--------------- \n')
    text_file.write('bin \t chA \t chB \t chC \t chD \n')
    text_file.write('observation \t {0:6.2f} \t {1:6.2f} \t {2:6.2f} \t {3:6.2f} \n'.format(a, b, c, d))
    text_file.write('------------------------------ \n')  
    text_file.write('bin \t chA \t chA \t chB \t chB \t chC \t chC \t chD \t chD \n')
    text_file.write('process sig \t bkg \t sig \t bkg \t sig \t bkg \t sig \t bkg \n')
    text_file.write('process 0 \t 1 \t 0 \t 1 \t 0 \t 1 \t 0 \t 1 \n')
    rate_string = 'rate'
    for i, rate in enumerate(signal_rate):

        rate_string =  rate_string+'\t {0:e} \t 1'.format(rate)
    text_file.write(rate_string+'\n')
    text_file.write('------------------------------ \n')  
    for ch in ['A','B','C','D']:
        text_file.write('NA_{0}\t rateParam \t ch{1} \t bkg \t {2:6.2f} \n'.format(data_year, ch, c))

    text_file.write('c1_{0}\t rateParam \t chA \t bkg \t {1:e} \n'.format(data_year, c1))
    text_file.write('c2_{0}\t rateParam \t chA \t bkg \t {1:e} \n'.format(data_year, c2))
    text_file.write('c1_{0}\t rateParam \t chB \t bkg \t {1:e} \n'.format(data_year, c1))
    text_file.write('c2_{0}\t rateParam \t chD \t bkg \t {1:e} \n'.format(data_year, c2))

#     #### uncertainties ####
    text_file.write('lumi\t lnN \t 1.025000 \t - \t 1.025000 \t - \t 1.025000 \t - \t 1.025000 \t - \n')
    text_file.write('signal_yield \t lnN \t 1.2 \t - \t 1.2 \t - \t 1.2 \t - \t 1.2 \t - \n')
    text_file.write('data_pred \t lnN \t - \t {0} \t - \t {1} \t - \t {2} \t - \t {3} \n'.format(1+unc[0], 1+unc[1], 1+unc[2], 1+unc[3]))



    text_file.close()


# significance optimization

# background prediction

# create datacard

# check correlation

## Distribution for event level variables in different binnings

# Roc curve for event level variables

# Data plot

In [ ]:
import importlib
importlib.reload(sys.modules['histo_utilities'])
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot
leg = rt.TLegend(0.5,0.80,0.90,0.90)

leg.SetTextSize(0.03)
leg.SetBorderSize(0)

leg.SetEntrySeparation(0.01)
c = rt.TCanvas('c','c', 800, 800)
# Plotting ncsc
start_t = time.time()

legend = {}
legend['data_oot'] = 'Data OOT region'
legend['data_intime'] = 'Data in-time region'
legend['mc_signal'] = 'signal MC in time'
legend['mc_intime'] = 'MC in-time background'
legend['mc_oot'] = 'MC OOT background'
legend['mc_bkg'] = 'QCD 50toInf background'




keys = ['data_intime','data_oot','mc_signal',]
keys = ['data_intime','mc_signal',]


h = {}
rt.gStyle.SetOptFit(1011)
# for i,k in enumerate(tree_bkg.keys()):
br = 1
for i,k in enumerate(keys):

#     if k == 'data_intime':
#         cond = nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < 200
        
#     else:
#         cond = np.ones(weight[k].shape, dtype=bool)
#     cond = np.logical_and(cond, jetMet_dPhiMin4[k]>0.25)
    if k == 'mc_signal':
        br = 1
    else: br = 1
#     if k == 'mc_background':br = 1
    bins = [50,0,1000]
    metcut = 200
    h[k] = create_TH1D( nCsc_JetMuonVetoCluster0p4_Me1112Veto[k][cond], axis_title=['N_{rechits}', 'Events'], name=k, binning=bins,weights = br*weight[k][cond])
    h[k].SetLineColor(std_color_list[i])
    print("weight",weight[k][0])
    leg.AddEntry(h[k], legend[k]+" (scaled)" if k == 'data_oot' else legend[k])
#     if k == 'mc_signal':
#         for i in range(h[k].GetNbinsX()):
#             s = 0.015*h[k].GetBinContent(i+1)
#             b = h['data_oot'].GetBinContent(i+1)
#             if s+b == 0:continue
#             sig = s/(s+b)**0.5
#             print(i,h[k].GetBinCenter(i+1),sig,s,b)
#             if sig < 1./100:
#                 print(i,sig)
#                 continue
#     h[k].SetMinimum(1)

#     r = h[k].Fit('expo', 'LRSQ+', '', 50,400)
#     print(r.Get())
#     print(r.Prob())
#     h[k].GetFunction("expo").SetLineColor(std_color_list[2])
#     h[k].GetFunction("expo").SetLineWidth(2)
    h[k].GetXaxis().SetLabelSize(0.04)
    h[k].SetMinimum(0.1)
    h[k].SetMaximum(10E6/2)
#     h[k+'met'].GetXaxis().SetLabelSize(0.02)
    if k == 'data_oot':
        scale = 1.0*h['data_intime'].GetBinContent(4)/h['data_oot'].GetBinContent(4)
#         for i in range(10):
#             print(1.0*h['data_intime'].GetBinContent(i+1)/h['data_oot'].GetBinContent(i+1))
        h[k].Scale(scale)
    if k[:4] == 'data':
        h[k].Draw('same E1')
    else:
        h[k].Draw("same hist")
#     h[k+'met'].DrawNormalized('same')
    

# c = make_ratio_plot([h['mc'],h['data']], fit = False, logy=True, in_tags = ["MC","Data"], ratio_bounds = [0,3])
c.SetRightMargin(0)
c.SetLogy()
leg.Draw()

c.Draw()
# print(time.time()-start_t)

In [ ]:
import importlib
importlib.reload(sys.modules['histo_utilities'])
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot
leg = rt.TLegend(0.5,0.80,0.90,0.90)

leg.SetTextSize(0.03)
leg.SetBorderSize(0)

leg.SetEntrySeparation(0.01)
c = rt.TCanvas('c','c', 800, 800)
# Plotting ncsc
start_t = time.time()


k = 'data_intime'

h = {}
rt.gStyle.SetOptFit(1011)

bins = [40,0,1000]
metcut = 200

# cscRechitClusterMaxChamber = {}
# cscRechitClusterNChamber = {}
# cscRechitClusterNStation = {}
# cscRechitClusterMaxStationRatio = {}
# cscRechitClusterMaxChamberRatio = {}

# h[k] = create_TH1D( cscRechitClusterMaxChamber[k], axis_title=['max chamber', 'Events'], name=k, binning=[200,-50,50])
h[k] = create_TH1D( cscRechitClusterMaxChamberRatio[k], axis_title=['max chamber ratio', 'Events'], name=k, binning=[20,0,1])
# h[k] = create_TH1D( cscClusterMet_dPhi[k], axis_title=['#Delta#phi(MET, cluster)', 'Events'], name=k, binning=[20,0,3.14])
# h[k] = create_TH1D( n_chamber[k], axis_title=['Number of Chambers', 'Events'], name=k, binning=[10,0,10])
# h[k] = create_TH1D( cscRechitClusterNStation[k], axis_title=['max station', 'Events'], name=k, binning=[5,0,5])
# print(cscRechitClusterMaxStationRatio[k])
# h[k] = create_TH1D( cscRechitClusterNStation[k], axis_title=['Number of station', 'Events'], name=k, binning=[20,0,10])


# h['1jet'].GetXaxis().SetLabelSize(0.04)
# h['1jet'].SetMinimum(0.1)
# h['1jet'].SetMaximum(10E6/2)

h[k].Draw('hist')


c.SetRightMargin(0)
c.SetLogy()

c.Draw()
# print(time.time()-start_t)

## significance

In [ ]:
start_t = time.time()
#ROC curve
c = rt.TCanvas('c','c', 800, 800)
leg = rt.TLegend(0.35,0.80,0.85,0.92)
leg.SetTextSize(0.022)
leg.SetBorderSize(0)

leg.SetEntrySeparation(0.01)

var = [nCsc_JetMuonVetoCluster0p4_Me1112Veto,]
name = ['nCsc_JetMuonVetoCluster0p4_Me1112Veto',]
legend = [ 'jet & muon veto + ME11/12 veto',]

bkg_k = 'data_oot'
sig_k = 'mc_signal'
br = 0.01

threshold = list(range(300))
threshold = np.array(list(range(2000)))
# print(threshold)
for i in range(len(var)):
    event_count = {}
    
    for k in ['data_oot','mc_signal']:
        event_count[k] = []
        for th in threshold:
            event_count[k].append(np.sum(weight[k][var[i][k]>th]))
        event_count[k] = np.array(event_count[k])
        if k == sig_k:
            event_count[k] = event_count[k]*br
    sig = event_count[sig_k]/np.sqrt(event_count[sig_k]+event_count[bkg_k])
    cond = event_count[sig_k]>0
    sig = sig[cond]
    ncsc = threshold[cond]
#     gr['bbbb'] = create_TGraph(eff_bkg,eff_sig,axis_title = ['#epsilon_{bkg}','#epsilon_{signal}'])
    h[sig_k+str(i)] = create_TGraph(ncsc, sig,  axis_title=['N_{rechits}', 'Significnace'])
#     h[sig_k+str(i)] = create_TGraph(ncsc, event_count[sig_k],  axis_title=['N_{rechits}', 'Event Yield'])

    h[sig_k+str(i)].SetLineWidth(2)
    h[sig_k+str(i)].SetMarkerColor(std_color_list[i])
    h[sig_k+str(i)].SetLineColor(std_color_list[i])
    h[sig_k+str(i)].GetXaxis().SetRangeUser(0, 3000)
    h[sig_k+str(i)].GetXaxis().SetLabelSize(0.04)

#     h[sig_k+str(i)].GetYaxis().SetRangeUser(0, 3)


    leg.AddEntry(h[sig_k+str(i)],sig_k+" "+legend[i])
    h[sig_k+str(i)].Draw('ac' if i == 0 else 'c')

#     h[sig_k+str(i)].Draw('ac' if i==0 else 'c')
#     gr['bbbb'].Draw('')




# c.SetLogy()
c.Draw()
# c.SaveAs('../plots/timing_studies/compare_timing_definition_ROC.png')

print(time.time()-start_t)


# 2d plot

In [ ]:
# Plotting ncsc
start_t = time.time()
c = rt.TCanvas('c','c', 1000, 800)
h = {}
# leg = rt.TLegend(0.50,0.75,0.97,0.93)
leg = rt.TLegend(0.58,0.70,0.94,0.85)
leg.SetTextSize(0.022)
# leg.SetTextFont(42)
leg.SetEntrySeparation(0.01)

i = 0
k = 'data_intime'

# h[k] = create_TH2D(np.column_stack((angle[k][cond],jetPt[k][cond][:,0])), axis_title = ['\Delta\phi (jet,met)','jet p_{T}[GeV]','Events'], binning=[80,-3.14,3.14,100,0,2000])
h[k] = create_TH2D(np.column_stack(( nCsc_JetMuonVetoCluster0p4_Me1112Veto[k], dphiMet_cluster[k])),
                   axis_title = ['N_{rechits}','\Delta\phi (cluster,met)','Events'], binning=[50,0,600,50,0,3.14])
h[k] = create_TH2D(np.column_stack(( nCsc_JetMuonVetoCluster0p4_Me1112Veto[k], jetMet_dPhiMin4[k])),
                   axis_title = ['N_{rechits}','min_{4jet}\Delta\phi (jet,met)','Events'], binning=[50,0,600,50,0,3.14])


h[k].GetXaxis().SetLabelSize(0.04)




h[k].SetLineColor(std_color_list[i])
# leg.AddEntry(h[k], k)
#     h[k].SetMaximum(10E5)
#     h[k].SetMinimum(1)
h[k].Draw('colz')
c.SetRightMargin(0.2)



# if setlog: 
c.SetLogz()
c.Draw()
print(time.time()-start_t)

# third

# ROC Curve

# 4th

# nCsc histogram

# event yield vs. ncsc

# significance